# How to use `unfold`

Author: [Romain Sacchi](romain.sacchi@psi.ch)

Github repo: https://github.com/romainsacchi/

## fold
Several `brightway` databases are "folded" in a data package.

In [16]:
from unfold import Fold
import bw2data, bw2calc
bw2data.projects.set_current("some BW2 project") # <-- name of the brightway project containing both the source database and the databases to fold

We run the LCA of 1 kWh from `market group for electricity, high voltage` in the region `WEU` from a prospective LCA database
generated by `premise` (IAM IMAGE, scenario SSP2-Base, year 2030).

In [19]:
act = [a for a in bw2data.Database("ecoinvent_image_SSP2-Base_2030")
      if a["name"] == "market group for electricity, high voltage" and a["location"] =="WEU"][0]
lca = bw2calc.LCA({act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))
lca.lci()
lca.lcia()
lca.score

0.32128633817019087

Now, we fold that database, along with another one, into a data package.

In [ ]:
f = Fold()

In [ ]:
f.fold()

In [ ]:
# alternatively, you can answer these questions directly by passing the following arguments
f.fold(
    package_name="a package name",
    package_description="some description",
    source="cutoff38",
    system_model="cut off",
    version="3.8",
    databases_to_fold=["ecoinvent_image_SSP2-Base_2025", "ecoinvent_image_SSP2-Base_2050"],
    descriptions=["some db", "some other db"],
)

## unfold
Here, we read the data package wee produced above, and we unfold and write the folded databases back into another `brightway` project.
It will write one database per scenario to your `brightway` project.

In [ ]:
from unfold import Unfold
import bw2data
bw2data.projects.set_current("some other BW2 project")

In [ ]:
u = Unfold("test.zip")

In [ ]:
u.unfold()

In [ ]:
# alternatively, you can answer these questions directly by passing the following arguments
u.unfold(
    dependencies={
        "ecoinvent 3.6 cutoff": "ecoinvent 3.6 cutoff",
        "biosphere3": "biosphere3"
    }
)

We test if we can find the same LCA score.

In [20]:
bw2data.projects.set_current("ammonia")
act = [a for a in bw2data.Database("ecoinvent_image_SSP2-Base_2030")
      if a["name"] == "market group for electricity, high voltage" and a["location"] =="WEU"][0]
lca = bw2calc.LCA({act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))
lca.lci()
lca.lcia()
lca.score

0.3212919009162858

In [22]:
(0.32128633817019087/0.3212919009162858)*100

99.99826863170873

We do, with a relative imprecision of 0.002%.

# unfold
## superstructure databases
`unfold` can also unfold [superstructure](https://link.springer.com/article/10.1007/s11367-021-01974-2) databases, to be used with [Activity Browser](https://github.com/LCA-ActivityBrowser/activity-browser).
It will write only one database, regardless of the number of scenarios.
It will also export a scenario difference spreadsheet to be read by Activity Browser.

In [ ]:
u.unfold(
    scenarios=[0, 1],
    dependencies={
        "cutoff38": "cutoff36",
        "biosphere3": "biosphere3"
    },
    superstructure=True # <-- NEW
)